# Project

https://www.kaggle.com/c/kktv-data-game-1711

Original Training Data: (92951083, 12)


internet_connection_type (7, object): [cellular, cellular 2G, cellular 3G, cellular 4G, offline, online but unknown, wifi]
<br>
platform (3, object): [Android, Web, iOS]
<br>

action_trigger (14, object): [api error, cast, error, interrupt, ..., program stopped, program stopped or enlarged-reduced, seek, video ended]

<br>
最後一個training data的使用者user ID為57158  (data-045.csv)
最後一個label data的使用者user ID為57158  (label-045.csv)


/home/user/data/ML_SVN/DataScience/project

In [ ]:
import sys  # handle command argument
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing   #for normalization
from sklearn.model_selection import cross_val_score  # for cross validation
from sklearn import linear_model
from sklearn import  metrics # confusion matrix
from sklearn.model_selection import train_test_split
import itertools #  for  plot_confusion_matrix
import pandas as pd
import os, sys,glob
from pathlib import Path
from datetime import datetime,date
    

In [ ]:
def process_time(stime):
    #'2017-11-24 7:59:25.334'
    time_slot=-1
    date1,time1= stime.split(' ')
    date1=[int(i) for i in date1.split('-')]   #[2017, 6, 8] Y/M/D
    time1=[float(i) for i in time1.split(':')]  #[15.0, 18.0, 25.334] H/M/S
    t=datetime(date1[0], date1[1],date1[2],int(time1[0]),int(time1[1])).time()

    t0=t.replace(hour=1,minute=0)
    t1=t.replace(hour=9,minute=0)
    t2=t.replace(hour=17,minute=0)
    t3=t.replace(hour=21,minute=0)
    #print(t)
    #print(t0)  #01:00:00

    if (t>=t0 and t<t1): #AM1:00~AM9:00
        time_slot=0
    elif (t>=t1 and t<t2):  #AM9:00~PM17:00
        time_slot=1
    elif (t>=t2 and t<t3): #PM17:00~PM9:00 
        time_slot=2
    else:  # >PM 9:00
        time_slot=3
    # weekday
    day=date(date1[0], date1[1],date1[2])
    weekday=day.weekday()   #the day of the week with Monday=0, Sunday=6
    week_of_year=day.isocalendar()[1]      #2017/08/21  ==> 34th
    #print('==>',week_of_year)
    if (week_of_year>=52):
        week_of_year=1  #last week of last year is 53. e.g. 2017/1/1 will retrun 53,

    
    return weekday,time_slot,week_of_year-1




In [ ]:
# load data


support=100

def read_csv(files):
    #column filter
    
    '''
    all column list:
    user_id,device_id,session_id,title_id,event_time,played_duration,
    action_trigger,platform,episode_number,series_total_episodes_count,internet_connection_type,is_trailer
    '''
    #olny use partial column
    cols=['user_id','event_time','played_duration','is_trailer']
    df=pd.read_csv(files,usecols=cols)
    return df



def preprocess_file(user_list,file,start_id):    
   
    df=read_csv(file)

    max_user_id=df.iloc[-1:, 0] # last row of column 0
    max_id=np.array(max_user_id, dtype=pd.Series)[0] # series to ndarray
    print(max_id)
    #print(df.shape)
    
    #should i should previous ...
    
    for i in range(start_id,max_id+1):
        user_df=df[df['user_id']==i]
        #print("i=",i,"#",user_df.shape)
        #n_events=user_df.shape[0] #number of row
        total_duration=0 
        w, slot = 33, 28;  # number of weeks:34,   # number of timeslot:28
        time_list = [[0 for x in range(slot)] for y in range(w)] 
       

        
        for row in user_df.itertuples(index=True):
            weekday,tslot,week_of_year=process_time(getattr(row, "event_time"))

            t=getattr(row, "played_duration")
            #print(week_of_year,weekday*4+slot)
            time_list[week_of_year][weekday*4+tslot]+=t #duration count in each timeslot
            #total_duration+=t
         
        
        #if duration > support , then set to 1 ,else set to 0
                
        for p in range(w):
            for s in range(slot):
                if (time_list[p][s]>support): 
                    time_list[p][s]=1
                else:
                    time_list[p][s]=0
        

        user_list.append(time_list)
        

    return  max_id   #we know the last one

def preprocess_Xdata(user_list,start,end):
    start_user_id=-1
    
    path = "public/" 
   
    for i in range(start,end+1):
        buf = "data-%03d.csv"%(i)
        buf=os.path.join(path,buf)
        print('procesing...',buf)
        start_user_id=preprocess_file(user_list,buf,start_user_id+1)

def preprocess_Ydata(DataSet):
    combine_label_data()
    src_file="all_label.csv"
    data= open(src_file,'r')

    lines=[]
    i=0
    for line in data:
        #print(line.strip().split(','))
        #print(line.strip().split(',')[1:]) 
        if (i>0): #skip the header line
            DataSet.append((line.strip().split(',')[1:]))  #remove the first column 'user id'
        i+=1

def combine_label_data():
    print("concatenate all files")
    path = "public/" 
    allFiles=[]
    total_files=45
    #allFiles = glob.glob(os.path.join(path,"label-*.csv"))
    for i in range(total_files):
        buf = "label-%03d.csv"%(i+1)
        buf=os.path.join(path,buf)
        allFiles.append(buf)
        #print('procesing...',buf) 
        
   
    df = pd.concat(map(pd.read_csv,allFiles))
    #print(df.shape)
    #print("save file")
    df.to_csv('all_label.csv', encoding='utf-8', index=False)
    print("save file all_label.csv Successfully!")

#write data
def save_data(data,output_file):
    print('write file....')
    path = Path(output_file)
    with open(output_file, 'w') as f:
        for i in data:
            #print(int(item[1]))  #[1] the probability of Spam. 
            f.write('%d\n' %i)
    return





## Processing Train_Y

In [ ]:
train_Y=[]
#combine_label_data()
preprocess_Ydata(train_Y)

# train_Y ready
print(train_Y[0])


## Processing Train_X and Test_X

In [ ]:
#cube: [id][week][slot]
def show_cube(data,start,len):
    print('---')
    for i in range(start,start+len):
        print('[i=',i,']')
        for j in data[i]:
            print(j)
        

train_X=[]
test_X=[]

# pre-processing  
preprocess_Xdata(train_X,1,45)  #data-001.csv ~ data-045.csv for training data
preprocess_Xdata(test_X,46,75)  #data-046.csv ~ data-075.csv for testing data


print("Finished")

In [ ]:
show_cube(train_X,0,2)  #display the  content from user id to user id + length 
show_cube(train_X,1251,2)
